In [1]:
import pandas as pd 
import os
import numpy as np

In [4]:
def get_all_input_paths(base_path):
    input_paths = []
    # Caminhar recursivamente pelas pastas
    for root, dirs, files in os.walk(base_path):
        if "imputed" in root:
            input_paths.append(root)
    return input_paths

# Obter os diretórios de experimentos
dirs = np.sort(get_all_input_paths('./imputation_Statistics/experiments'))
experiment_dirs = []
for experiment in dirs:
    base = experiment.split('/')
    experiment_dirs.append("/".join(base[:3]))
experiment_dirs = list(np.unique(experiment_dirs))

In [ ]:
window_sizes = ['48', '512', '1024']
rates = ['10', '20', '30']


consolidated_results = {}

for experiment_dir in experiment_dirs:
    experiment_name = os.path.basename(experiment_dir)
    for window in window_sizes:
        df_list = []
        for rate in rates:
            base = os.path.join(experiment_dir, rate, window)
            csv_path = os.path.join(base, 'test_result.csv')
            
            # Verificar se o arquivo existe
            if not os.path.exists(csv_path):
                print(f"Arquivo não encontrado: {csv_path}")
                continue
            
            # Ler o CSV
            df_result = pd.read_csv(csv_path)
            
            # Processar a coluna 'serie'
            df_result['serie'] = df_result['serie'].apply(lambda x: "_".join(x.split('_')[1:]))
            
            # Agrupar por 'serie' e calcular a média
            df_result = df_result.groupby('serie').mean()
            
            # Renomear as colunas para incluir a taxa
            df_result = df_result.add_suffix(f'_{rate}')
            
            # Adicionar ao lista
            df_list.append(df_result)
        
        if df_list:
            # Concatenar os DataFrames horizontalmente com base no índice 'serie'
            df_combined = pd.concat(df_list, axis=1)
            
            # Resetar o índice para ter 'serie' como uma coluna
            df_combined = df_combined.reset_index()
            
            # Adicionar uma coluna para identificar o experimento e a janela
            df_combined['experiment'] = experiment_name
            df_combined['window_size'] = window
            
            # Reorganizar as colunas para ter 'experiment' e 'window_size' no início
            cols = ['experiment', 'window_size', 'serie'] + [col for col in df_combined.columns if col not in ['experiment', 'window_size', 'serie']]
            df_combined = df_combined[cols]
            
            # Armazenar no dicionário com chave composta
            key = f"{experiment_name}_{window}"
            consolidated_results[key] = df_combined

# Exemplo de como acessar e visualizar os resultados consolidados
for key, df in consolidated_results.items():
    print(f"Resultados Consolidated para {key}:")
    print(df.head())
    print("\n")

# (Opcional) Exportar os resultados consolidados para arquivos CSV ou LaTeX
# Exportar para CSV
for key, df in consolidated_results.items():
    output_csv = f"consolidated_results_{key}.csv"
    df.to_csv(output_csv, index=False)
    print(f"Arquivo CSV gerado: {output_csv}")

# (Opcional) Exportar para LaTeX
for key, df in consolidated_results.items():
    output_latex = f"consolidated_results_{key}.tex"
    df.to_latex(
        output_latex,
        index=False,
        float_format="%.2e",
        caption=f"Resultados Consolidated para {key}",
        label=f"tab:{key}",
        column_format='|l|l|l|' + 'S[table-format=1.2e+3]|S[table-format=1.2e+3]|S[table-format=1.2e+3]|S[table-format=1.2e+3]|',
        escape=False
    )
    print(f"Arquivo LaTeX gerado: {output_latex}")

Resultados Consolidated para experiments_48:
    experiment window_size               serie  pred_window_10  \
0  experiments          48       AirPassengers    7.830529e+01   
1  experiments          48        Bike Sharing    2.207125e+09   
2  experiments          48  Birmingham Parking    3.966937e+05   
3  experiments          48           Confirmed    4.961572e-02   
4  experiments          48              Deaths    7.960137e-02   

   best_window_10      worst_10  full_serie_10  pred_window_20  \
0        2.537825  1.133201e+02       2.537825       30.795082   
1       13.128226  2.209512e+09      19.078473       77.700516   
2       11.047865  3.967119e+05      16.871853      107.876953   
3        0.039859  1.518544e+05       0.039859        0.053793   
4        0.072860  6.893301e+03       0.072860        0.090070   

   best_window_20      worst_20  full_serie_20  pred_window_30  \
0        3.456699  2.540694e+02       3.456699        7.285635   
1       16.878675  6.702082e+